In [21]:
from bs4 import BeautifulSoup
import requests
# import time, os
import re
# import urllib
# import json
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import numpy as np

time: 854 µs
In [21] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 130.92 MiB


In [22]:
# ipython autotime
%load_ext autotime

# ipython_memory_usage
import ipython_memory_usage 
%ipython_memory_usage_start   

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


'memory profile enabled'

time: 3.57 ms
In [22] used 0.0039 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 130.93 MiB


In [23]:
start, end = 2256006, 2256009



time: 698 µs
In [23] used 0.0195 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 130.95 MiB


In [24]:
allf = []
for id in range(start, end):
    site = 'https://get61.com/hand_detail?handid={}'
    page = requests.get(site.format(id)).text
    soup = BeautifulSoup(page, 'html5lib')

    cardre = re.compile('var play11')
    cards = soup.find(text=cardre)
    a = cards.split(';')
    a
    b = a[0].split(' ')
    rounds = [i.split(' ') for i in a]
    if rounds[41][-1] == '"NA"':
        continue
    else:

        plays = [rounds[i][3].replace('"', '') for i in range(30)]

        handa = plays[:6]
        handb = plays[6:12]
        handc = plays[12:18]
        handd = plays[18:24]
        hande = plays[24:32]

        nameslist = [row.split('=') for row in a[36:41]]
        scores = [int(rounds[i][3].replace('"', '')) for i in range(41, 46)]
        names = [nameslist[row][1].strip().replace('"', '') for row in range(5)]
        leads = [int(rounds[i][3].replace('"', '')) for i in range(30, 36)]

        blind1, blind2 = rounds[48][-1].replace('"',''), rounds[49][-1].replace('"','')
        bury1, bury2 = rounds[46][-1].replace('"',''), rounds[47][-1].replace('"','')

        call_ace = rounds[50][-1].replace('"','')
        ace_suit = rounds[53][-1].replace('"','')
        is_leaster = rounds[51][-1].replace('"','')

        player1 = [names[0]] + handa + [scores[0]]
        player2 = [names[1]] + handb + [scores[1]]
        player3 = [names[2]] + handc + [scores[2]]
        player4 = [names[3]] + handd + [scores[3]]
        player5 = [names[4]] + hande + [scores[4]]
        df = pd.DataFrame([player1, player2, player3, player4, player5], columns = ['name', 1, 2,3, 4, 5, 6, 'score'])
        
        df['handid'] = id
        df['picker'] = df['score'].apply(lambda x: 'picker' if x == max(scores, key=abs) else 'partner' if np.sign(x) == np.sign(max(scores,                                               key=abs)) else 'no')
        df['alone'] = df['picker'].apply(lambda x: 'no' if 'partner' in df.values else 'yes')
        blind1, blind2 = rounds[48][-1].replace('"',''), rounds[49][-1].replace('"','')
        df['position'] = [0, 1, 2, 3, 4]
        df['lead0'] = leads[0]
        df['lead1'] = leads[1]
        df['lead2'] = leads[2]
        df['lead3'] = leads[3]
        df['lead4'] = leads[4]
        df['lead5'] = leads[5]
        df['pick_position'] = df.apply(lambda x: 0 if x['position'] == x['lead0'] else x['position'] + (5 % x['lead0']) if x['position'] < x['lead0'] else x['position'] - x['lead0'], axis=1)
        df['rd0pos'] = df.apply(lambda x: 0 if x['position'] == x['lead0'] else 4 if x['lead0'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead0']) if x['position'] < x['lead0'] else x['position'] - x['lead0'], axis=1)
        df['rd1pos'] = df.apply(lambda x: 0 if x['position'] == x['lead1'] else 4 if x['lead1'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead1']) if x['position'] < x['lead1'] else x['position'] - x['lead1'], axis=1)
        df['rd2pos'] = df.apply(lambda x: 0 if x['position'] == x['lead2'] else 4 if x['lead2'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead2']) if x['position'] < x['lead2'] else x['position'] - x['lead2'], axis=1)
        df['rd3pos'] = df.apply(lambda x: 0 if x['position'] == x['lead3'] else 4 if x['lead3'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead3']) if x['position'] < x['lead3'] else x['position'] - x['lead3'], axis=1)
        df['rd4pos'] = df.apply(lambda x: 0 if x['position'] == x['lead4'] else 4 if x['lead4'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead4']) if x['position'] < x['lead4'] else x['position'] - x['lead4'], axis=1)
        df['rd5pos'] = df.apply(lambda x: 0 if x['position'] == x['lead5'] else 4 if x['lead5'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead5']) if x['position'] < x['lead5'] else x['position'] - x['lead5'], axis=1)

        df['blind_1'] = blind1
        df['blind_2'] = blind2

        df['bury1'] = bury1
        df['bury2'] = bury2

        df['call_ace'] = call_ace
        df['ace_suit'] = ace_suit
        df['leaster'] = is_leaster
        allf.append(df)

time: 5.36 s
In [24] used 0.6797 MiB RAM in 5.47s, peaked 0.00 MiB above current, total RAM usage 131.62 MiB


In [25]:
dftot = pd.concat(allf)

time: 8.01 ms
In [25] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 131.62 MiB


In [26]:
dftot

,name,1,2,3,4,5,6,score,handid,picker,...,rd3pos,rd4pos,rd5pos,blind_1,blind_2,bury1,bury2,call_ace,ace_suit,leaster
0,tvretired,13D,8H,11D,12C,12S,10H,16,2256006,picker,...,0,0,0,13D,11D,14C,10C,,,
1,david cole,10D,7H,14S,11C,13H,14H,-4,2256006,no,...,1,1,1,13D,11D,14C,10C,,,
2,merchantnater,13C,9H,8S,7D,11S,9S,-4,2256006,no,...,2,2,2,13D,11D,14C,10C,,,
3,JimBobWilly,7C,8C,13S,11H,14D,12H,-4,2256006,no,...,3,3,3,13D,11D,14C,10C,,,
4,jordan mariposas,9C,12D,10S,8D,9D,7S,-4,2256006,no,...,4,4,4,13D,11D,14C,10C,,,
0,blueislandgirl,12C,8D,14D,11H,9C,7C,6,2256007,picker,...,1,0,1,9C,8D,10C,14H,,,
1,island,13D,10D,11S,10S,7H,8H,-3,2256007,no,...,2,1,2,9C,8D,10C,14H,,,
2,Schneider,12D,13C,8C,7S,8S,9S,-3,2256007,no,...,3,2,3,9C,8D,10C,14H,,,
3,barry g,12S,9H,13H,14S,14C,10H,-3,2256007,no,...,4,3,4,9C,8D,10C,14H,,,
4,CTWIFL,7D,11D,12H,13S,9D,11C,3,2256007,partner,...,0,4,0,9C,8D,10C,14H,,,


time: 28 ms
In [26] used 0.0469 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 131.67 MiB


In [27]:
#dftot.to_pickle('2268010_2268200')
#dftot.to_pickle('2268200_2268600')
#dftot.to_pickle('2268600_2270000')
#dftot.to_pickle('2270000_2272000')
#dftot.to_pickle('2272000_2274000')
#dftot.to_pickle('2274000_2277000')
#dftot.to_pickle('2277000_2280000')
#dftot.to_pickle('2280000_2282000')
#dftot.to_pickle('2282000_2284000')
#dftot.to_pickle('2284000_2285000')
#dftot.to_pickle('2285000_2287000')
#dftot.to_pickle('2293000_2296000')
#dftot.to_pickle('2296000_2397000')

time: 568 µs
In [27] used 0.0195 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 131.69 MiB


In [28]:
dftot.to_pickle('2256006_2256009')

time: 2.8 ms
In [28] used 0.0039 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 131.70 MiB
